You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets' folder:
- __Office addresses__ are currently saved in 'office_addresses.csv'. If the value for office is 'NaN', then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

In [106]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


## I. Import data 📩

Firstly, we will read from multiple sources and store them to temporary dataframe

In [107]:
import pandas as pd

In [108]:
employee_address_df = pd.read_excel('./datasets/employee_information.xlsx',
                                    sheet_name='employee_addresses',
                                    index_col='employee_id')
employee_address_df.head()

,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
employee_id,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
H8K0L6,Siff,Tara,GB,London,Baker Street,221
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [109]:
contact_columns = ["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"]

# The sheet doesn't have header, so we need to specify the column names, and not setting the first row as header
employee_contact_df = pd.read_excel('./datasets/employee_information.xlsx',
                                    sheet_name='emergency_contacts',
                                    header=None,
                                    names=contact_columns,
                                    index_col='employee_id')
employee_contact_df.head()

,last_name,first_name,emergency_contact,emergency_contact_number,relationship
employee_id,,,,,
A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [110]:
# We import the office's information
office_addresses_df = pd.read_csv('./datasets/office_addresses.csv')
print(office_addresses_df.isna().any())
office_addresses_df

office                  False
office_country          False
office_city             False
office_street           False
office_street_number    False
dtype: bool


,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [111]:
# Import the employee's roles
employee_roles_df = pd.read_json('./datasets/employee_roles.json',orient='index')
# Name the index column
employee_roles_df.index.name = 'employee_id'
employee_roles_df

,title,monthly_salary,team
employee_id,,,
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


## II. Proceed to merge data

We merge the 2 tables to form the Employee Information DataFrame.\
However, the columns about employees' first name and last name are different between the 2 tables.

In [112]:
# Rename the columns in the employee_address_df dataframe
employee_address_df.rename(columns={'employee_first_name': 'first_name', 'employee_last_name': 'last_name'}, inplace=True)
employee_information = employee_address_df.merge(employee_contact_df, on=['employee_id', 'first_name', 'last_name'], how='left')
employee_information.head()

,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship
employee_id,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother
H8K0L6,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife


*👉 We can see that only Hunman Jax doesn't have emergy contact*

In [113]:
# Merge employee_information_df and employee_roles_df on the index
employee_information_roles_df = employee_information.merge(employee_roles_df, on='employee_id', how='left')
employee_information_roles_df.head()

,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team
employee_id,,,,,,,,,,,,
A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership
H8K0L6,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership
G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales
M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations


In [114]:
# Similarly, office_df contains different column names with employee_df
# Therefore, we must rename the columns in the office_addresses_df dataframe
employee_information_roles_df.reset_index(inplace=True)
employee_final = employee_information_roles_df.merge(
    office_addresses_df, 
    left_on='employee_country', 
    right_on='office_country',
    how='left')
employee_final

,employee_id,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations,NaN,NaN,NaN,NaN,NaN


# III. Cleaning data

In [115]:
contact_cols = ['emergency_contact','emergency_contact_number',	'relationship']

In [116]:
# Loop through columns that start with "office"
for column in employee_final.columns:
    if column.startswith('office'):
        employee_final[column].fillna('Remote', inplace=True)
employee_final

C:\Users\Phu\AppData\Local\Temp\ipykernel_1400\2769778809.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Remote' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  employee_final[column].fillna('Remote', inplace=True)


,employee_id,last_name,first_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations,Remote,Remote,Remote,Remote,Remote


# III. Subsetting and indexing

In [117]:
# Subsetting
columns = ["employee_id", "first_name", "last_name", "employee_country", 
                 "employee_city", "employee_street", "employee_street_number", 
                 "emergency_contact", "emergency_contact_number", "relationship", 
                 "monthly_salary", "team", "title", "office", "office_country", 
                 "office_city", "office_street", "office_street_number"]
employee_final = employee_final[columns]
# Setting index
employee_final.set_index('employee_id', inplace=True)
employee_final.head()

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
